In [10]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch
from tqdm import tqdm  # 진행률 표시를 위한 tqdm 라이브러리

mirrored_strategy = tf.distribute.MirroredStrategy()
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.10.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.9.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [11]:
filename = 'prompt.txt'
f = open(filename, 'r')
lines = f.readlines()
print(lines)
print('-' * 10)

['아침 일과 시작하기\n', '책 읽기의 즐거움\n', '가족과 함께 하는 저녁 식사\n', '친구와의 영화 관람\n', '주말 동안의 여행 계획\n', '정원 가꾸기의 효과\n', '요리를 통한 스트레스 해소\n', '자전거 타기의 장점\n', '집안 정리 정돈 팁\n', '개인 운동 루틴 유지하기\n', '취미 활동의 중요성\n', '새로운 언어 배우기\n', '음악 연주와 그 영향\n', '사진 촬영을 통한 추억 만들기\n', '지역 사회 봉사의 보람\n', '패션과 개인 스타일 표현\n', '휴일을 활용한 가족 모임\n', '건강한 식생활을 위한 조리법\n', '애완동물과 보내는 시간\n', '독서 클럽에서의 토론\n', '도서관 방문의 장점\n', '집에서의 필름 감상\n', '커피 한 잔과의 여유\n', '일기 쓰기의 치유 효과\n', '홈 베이킹의 즐거움\n', '직접 만든 공예품\n', '새로운 요리법 시도하기\n', '지역 공연 관람\n', '자연 속에서의 하이킹\n', '가구 리폼 프로젝트\n', '아이와 함께하는 공원 나들이\n', '새로운 스포츠 도전\n', '개인 기록의 개선\n', '집에서의 요가 세션\n', '반려동물과의 산책\n', '가족 앨범 만들기\n', '매일의 명상 시간\n', '가벼운 아침 조깅\n', '주말 브런치 준비\n', '일상의 소소한 행복 찾기\n', '스마트폰 없는 하루\n', '책상 정리의 중요성\n', '옷장 정리하기\n', '새벽 시간의 고요함\n', '오후의 티타임\n', '자기 전 명상\n', '집안 식물 가꾸기\n', '근처 박물관 탐방\n', '집에서의 칵테일 파티\n', '커뮤니티 클래스 참여\n', '셀프 케어 루틴\n', '장거리 걷기의 이점\n', '텃밭 가꾸기\n', '개인적인 스타일 개발\n', '책 추천 공유\n', '로컬 마켓 쇼핑\n', '장기 게임의 즐거움\n', '가족과의 보드게임 밤\n', '디지털 디톡스\n', '가까운 도시 탐험\n', '수제 비누 만들기\n', '로컬 아

In [13]:

with open('perplexity.txt', 'w') as f:  #file내용 clear
    f.write('')

for line in lines:
    line = line.strip()
    print(f"prompt : {line}")

    input_text = line
    input_ids = tokenizer.encode(input_text, return_tensors='tf')
    #term = 3

    last_index = input_ids.shape[-1]
    nlls = []
            
    with mirrored_strategy.scope():
        for _ in tqdm(range(100)):  # 최대 100번 반복
            if input_ids.shape[-1] >= 100:
                break  # input_ids의 길이가 100 이상이면 반복 중단
            
            #print("================================")
            #print(f"문장의 토큰 개수 : {input_ids.shape[-1]}")
            #set 나눠서 green set 로짓을 촉진하기
            gamma = 1.0
            delta = 0.0
            Logits = model(input_ids).logits
            #print(f"original Logits : {Logits}")
            
            #seed 생성
            seed = input_ids[-1] % Logits.shape[-1]
            #print(f"seed : {seed[-1]}")
            np.random.seed(seed[-1])

            #random green list 설정
            indices = np.arange(Logits.shape[-1]) #0~51999
            green_list_size = int(Logits.shape[-1] * gamma)  #25600
            green_list = np.random.choice(indices, green_list_size, replace=False)
            #print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

            #--여기부터 probability 구하기
            # green logit과 red logit의 값을 담을 리스트 생성
            green_logits = tf.zeros(green_list_size)
            red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

            #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
            Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
            Logits_num = Logits.numpy()
            for i in green_list :
                Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
            Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
            #print(f"updated Logits : {Logits}")
            
            # softmax 계산을 위한 분모
            denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  
            #print(f"denominator : {denominator}") #25600개의 green list

            # Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산), softmax_tensor = probability
            softmax_tensor = Logits.numpy()
            softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
            softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
            #print(softmax_tensor.shape)
            #print(f"Probabilities : {softmax_tensor}")

            # 데이터 타입 확인 및 조정
            add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
            #print(f"updated Logits Max value : {add_token_id}")
            
            # Calculate perplexity
            # 참고 https://huggingface.co/docs/transformers/main/ko/perplexity
            selected_probability = softmax_tensor[0, last_index-1, add_token_id]
            nlls.append(np.log(selected_probability))
            
            #print(softmax_tensor[0, last_index-1, add_token_id])

            # 텐서 연결
            add_token_id = tf.reshape(add_token_id, [1,1])
            add_token_id = tf.cast(add_token_id, input_ids.dtype)
            input_ids = tf.concat([input_ids, add_token_id], axis=-1)
            next_token = tokenizer.decode(input_ids.numpy()[0,:])
            #print(f"next token : {next_token}")
            
            last_index = input_ids.shape[-1]
            #term = 3
        final_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(final_token)

        # Calculate perplexity
        perplexity = tf.exp(-tf.reduce_mean(nlls))
        print("Perplexity: ", perplexity.numpy())
        
        perplexity_str = str(perplexity.numpy())  # perplexity를 문자열로 변환
        with open('perplexity.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
            f.write(perplexity_str + '\n')

prompt : 아침 일과 시작하기


 96%|█████████▌| 96/100 [00:24<00:01,  3.97it/s]


아침 일과 시작하기 전에 미리 준비해두면 좋다.
아침식사는 아침식사 후 바로 먹는 것이 좋다.
아침식사는 아침식사 후 바로 먹는 것이 좋다.
아침식사는 아침식사 후 바로 먹는 것이 좋다.
아침식사는 아침식사 후 바로 먹는 것이 좋다.
아침식사는 아침식사 후 바로 먹는 것이 좋다.
아침식사는 아침식사 후 바로 먹는 것이 좋다.
아침식사는 아침식사 후 바로 먹는 것이 좋다.
아침식사는 아침
Perplexity:  2.0498257
prompt : 책 읽기의 즐거움


 95%|█████████▌| 95/100 [00:23<00:01,  4.03it/s]


책 읽기의 즐거움도 함께 느낄 수 있다.
이번 전시는 ‘책 읽는 즐거움’을 주제로 한 ‘책 읽는 즐거움’ 전시를 비롯해 ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움
Perplexity:  2.04693
prompt : 가족과 함께 하는 저녁 식사


...

지역사회와의 소통 강화와 지역경제 활성화를 위해 노력하겠다"고 말했다.</d> 서울중앙지검 특수1부(부장검사 이원석)는 이날 오전 10시30분부터 서울 서초구 서울중앙지검 청사에 수사관들을 보내 컴퓨터 하드디스크와 관련 서류 등을 확보했다.
검찰은 지난달 30일 압수수색을 통해 확보한 컴퓨터 하드디스크와 관련 자료를 분석 중이다.
검찰은 압수물 분석을 통해 확보한 자료를 토대로 조만간 관련자들을 소환해 조사할 방침이다.
검찰은 지난달 30일 조 전 장관
Perplexity:  2.7735355
prompt : 가족과의 소중한 대화


 96%|█████████▌| 96/100 [00:21<00:00,  4.53it/s]


가족과의 소중한 대화에서 많은 것을 배웠다”며 “앞으로도 다양한 사회공헌활동을 통해 지역사회와 함께하는 기업이 되도록 노력하겠다”고 말했다.</d> 서울중앙지검 특수1부(부장검사 이원석)는 이날 오전 10시30분부터 서울 강남구 삼성동 코엑스 오디토리움에서 열린 '2017 삼성바이오로직스 분식회계 사건' 공판에서 삼성바이오로직스 회계 담당 임직원들을 불러 조사하고 있다.
검찰은 삼성바이오로직스 회계
Perplexity:  2.876835
prompt : 개인 발전을 위한 목표 설정


 95%|█████████▌| 95/100 [00:21<00:01,  4.52it/s]

개인 발전을 위한 목표 설정과 함께 지역사회의 다양한 의견을 수렴해 지역사회의 발전을 위한 다양한 방안을 모색해 나갈 계획이다.
한편 이번 워크숍은 지역사회의 다양한 의견을 수렴해 지역사회의 발전을 위한 다양한 방안을 모색하기 위해 마련됐다.</d> 서울중앙지검 특수1부(부장검사 이원석)는 이날 오전 10시30분부터 서울 강남구 삼성동 코엑스 오디토리움에서 열린 '2017 삼성바이오로직스 분식회계 사건' 공판에서 삼성바이오로직
Perplexity:  3.3996139


In [3]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset
import tensorflow as tf
from tqdm import tqdm

In [4]:

# 모델과 토크나이저 설정
model_id = "gpt2"
model = TFGPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2Tokenizer.from_pretrained(model_id)

# 데이터셋 로딩
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="tf")

# 설정 값
max_length = model.config.n_positions
stride = 512

nlls = []
input_shape = encodings.input_ids.shape[1]
for i in tqdm(range(0, input_shape, stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, input_shape)
    trg_len = end_loc - begin_loc

    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = tf.identity(input_ids)

    # 마스킹: 타겟 토큰의 길이 만큼만 계산하고 나머지는 -100 (ignore index)
    masks = tf.concat([
        tf.fill([target_ids.shape[0], target_ids.shape[1] - trg_len], -100),
        tf.fill([target_ids.shape[0], trg_len], 0)
    ], axis=1)
    target_ids = tf.where(masks == -100, -100, target_ids)

    with tf.GradientTape() as tape:
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs.loss * trg_len

    nlls.append(neg_log_likelihood)

# Perplexity 계산
ppl = tf.exp(tf.reduce_sum(tf.stack(nlls)) / input_shape)

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Using the latest cached version of the dataset since wikitext couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'wikitext-2-raw-v1' at /home/undergrad/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/0.0.0/b08601e04326c79dfdd32d625aee71d232d685c3 (last modified on Mon Apr 22 20:57:14 2024).
Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors
100%|██████████| 562/562 [01:25<00:00,  6.58it/s]


In [5]:
print(ppl)

tf.Tensor(888.30756, shape=(), dtype=float32)


In [ ]:

# 모델과 토크나이저 설정
model_id = "skt/kogpt2-base-v2"
model = TFGPT2LMHeadModel.from_pretrained(model_id)
tokenizer = GPT2Tokenizer.from_pretrained(model_id)

# 데이터셋 로딩
test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="tf")

# 설정 값
max_length = model.config.n_positions
stride = 512

nlls = []
input_shape = encodings.input_ids.shape[1]
for i in tqdm(range(0, input_shape, stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, input_shape)
    trg_len = end_loc - begin_loc

    input_ids = encodings.input_ids[:, begin_loc:end_loc]
    target_ids = tf.identity(input_ids)

    # 마스킹: 타겟 토큰의 길이 만큼만 계산하고 나머지는 -100 (ignore index)
    masks = tf.concat([
        tf.fill([target_ids.shape[0], target_ids.shape[1] - trg_len], -100),
        tf.fill([target_ids.shape[0], trg_len], 0)
    ], axis=1)
    target_ids = tf.where(masks == -100, -100, target_ids)

    with tf.GradientTape() as tape:
        outputs = model(input_ids, labels=target_ids)
        neg_log_likelihood = outputs.loss * trg_len

    nlls.append(neg_log_likelihood)

# Perplexity 계산
ppl = tf.exp(tf.reduce_sum(tf.stack(nlls)) / input_shape)